# Arbitrage and bond markets

In [1]:
import pandas as pd
import numpy as np

## Bootstrapping

In [2]:
A = [97.5, 0, 0.5, 100]
B = [95, 0, 1.0, 100]
C = [955, 0.025, 1.5, 1000]
D = [1000, 0.0575, 2.0, 1000]

bonds = np.row_stack((A, B, C, D))

In [3]:
df = pd.DataFrame(bonds, 
        columns = ['price', 'coupon_rate', 'maturity','face'], 
        index=['A','B','C','D'])
df['period'] = df.maturity*2
df=df.reset_index()
df.index = df.index+1
df.columns = ['bond', 'price', 'coupon_rate', 'maturity','face', 'period']
df

,bond,price,coupon_rate,maturity,face,period
1,A,97.5,0.0000,0.5,100.0,1.0
2,B,95.0,0.0000,1.0,100.0,2.0
3,C,955.0,0.0250,1.5,1000.0,3.0
4,D,1000.0,0.0575,2.0,1000.0,4.0


### Functions to generalize bootstrapping and pricing

In [4]:
def bootstrap(d, nop):
    ''' 
    d: a dataset with prices, face value, coupon_rate, period of maturity
    nop: compounding period (ie., 1 = annual, 2=semiannual)
    '''
    spot_rates = []
    for t in d.index:
        p      = d.loc[t,'price']
        face   = d.loc[t,'face']
        cr     = d.loc[t,'coupon_rate']
        period = int(d.loc[t,'period'])
        coupon = face*cr/nop
        cf_maturity = face + coupon

        pv_coupons = 0.0
        for i in range(period-1):
            pv_coupons = pv_coupons + coupon / (1+spot_rates[i])**(i+1)
        x = p - pv_coupons
        z = (cf_maturity / x )**(1/period) - 1
        spot_rates.append(z)
    return spot_rates
spot_curve = bootstrap(df,2)
spot_curve

[0.02564102564102555,
 0.025978352085153977,
 0.028390767355670476,
 0.02882263912764249]

In [5]:
def spot_pricer(cfs, spot):
    ''' 
    cfs:  list of cash flows at each period
    spot: list of spot rates at each period
    cfs should be the same length as spot
    '''
    cfs = np.array(cfs)
    discount_factors = np.array([1/(1+spot[i])**(i+1) for i in range(len(spot))])
    return cfs @ discount_factors
spot_pricer([0,100,0,1100], spot_curve)

1076.8171999459971

In [6]:
# Check that the prices are consistent with the spot curve

# pick a bond
BOND_ID = 'A'
bond_dict = {'A': 1, 'B':2, 'C':3, 'D':4}
def cf_generator(bond_id):
    i = bond_dict[bond_id]
    nop = 2
    face   = df.loc[i,'face']
    cr     = df.loc[i,'coupon_rate']
    period = int(df.loc[i,'period'])
    coupon = face*cr/nop
    cf_maturity = face + coupon
    cfs =  np.zeros(len(spot_curve))
    for i in range(period-1):
        cfs[i] = coupon
    cfs[period-1] = cf_maturity
    print('The bond cash flows for bond ' + bond_id + ' are: ', cfs)
    return cfs
cfs = cf_generator(BOND_ID)

p = spot_pricer(cfs.tolist(),spot_curve)
print('The fair price for ' + BOND_ID + f' is: ${p:,.2f}')


The bond cash flows for bond A are:  [100.   0.   0.   0.]
The fair price for A is: $97.50


### Arbitrage and replicating portfolio


Example #1 (2-period bond)

In [7]:
spot_pricer([15,115,0,0], spot_curve)

123.87500000000003

Example #2 (4-period bond)

In [8]:
# Target bond cash-flows
cfs = np.array([0,100,0,1100])
cfs

array([   0,  100,    0, 1100])

In [9]:
# matrix of cash flows for bonds A-D
CF = np.zeros((len(df), len(df)))
for j, id in enumerate(['A','B','C','D']):
    print(j)
    print(id)
    CF[j] = cf_generator(id)
CF

0
A
The bond cash flows for bond A are:  [100.   0.   0.   0.]
1
B
The bond cash flows for bond B are:  [  0. 100.   0.   0.]
2
C
The bond cash flows for bond C are:  [  12.5   12.5 1012.5    0. ]
3
D
The bond cash flows for bond D are:  [  28.75   28.75   28.75 1028.75]


array([[ 100.  ,    0.  ,    0.  ,    0.  ],
       [   0.  ,  100.  ,    0.  ,    0.  ],
       [  12.5 ,   12.5 , 1012.5 ,    0.  ],
       [  28.75,   28.75,   28.75, 1028.75]])

In [10]:
# Solve the system of equations
portfolio = cfs @ np.linalg.inv(CF)
portfolio

array([-0.3036167 ,  0.6963833 , -0.03036167,  1.06925881])

In [11]:
# Check that replicating portfolio delivers same cash flows
repl_cfs = portfolio @ CF 
repl_cfs.astype(int)

array([   0,  100,    0, 1100])

In [12]:
# Price of replicating portfolio
repl_price = portfolio @ df.price
repl_price

1076.8171999459971

In [13]:
profit = repl_price-1050
profit

26.817199945997118